# Mean-Variance Portfolio Optimization with a Risk-Free Asset
This script performs an event study analysis, following Kaplanski and Levi (2010) around specific events, using a CAPM model for expected returns in Python.

**Source:** Essentials of Financial Economics  
**Authors:** Michael Donadelli, Michele Costola, Ivan Gufler  
**Date:** May 8, 2025

## 0. Preliminaries

This script performes the regression-based event study around large aviation disasters (more than 150 casualties) between 1972 and 2007. The test asset is the Transport industry from 49 Industry portfolio retrieved from Fama and French library.

## 1. Import libraries

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta # Import timedelta specifically
import statsmodels.api as sm
import matplotlib.pyplot as plt

## 2. Settings

We include a number $maxlag$ of lagged returns as control. The number of days after the event included in the regression is controlled by $fD$.  

In [3]:
maxlag = 3  # Maximum lag for lagged returns in the regression
cutoff = 150  # Cutoff value for filtering events (based on column 13 in Events.xlsx)
fD = 3  # Number of future days to create event dummies for
cons = 1 # Flag for including a constant in the regression (1 for include) - Note: statsmodels OLS adds constant separately
America = 1  # Flag to filter events by Zone (1 for America, 0 for All)


## 3. Load and Prepare Data

In [6]:
# Import industry portfolio returns from an Excel file
Ret = pd.read_excel("../Data/IndustryPortfolios.xlsx")
# Extract dates from the first column and convert to datetime objects
DatesReturn = pd.to_datetime(Ret.iloc[:, 0], format='%Y%m%d')

# Import events data from an Excel file
Events = pd.read_excel('../Data/Events.xlsx')

# Filter events based on the America flag and casualty cutoff
if America == 0:
    # Filter for all events above the cutoff (assuming casualties are in column 12, event date in column 3)
    Events = Events[Events.iloc[:, 12] > cutoff].iloc[:, 3]
else:
    # Filter for events in Zone 1 (America, assuming Zone is in the last column) above the cutoff
    Events = Events[(Events.iloc[:, 12] > cutoff) & (Events["Zone"] == 1)].iloc[:, 3]

# Extract returns data (assuming it's in the 14th column, index 13)
Ret = Ret.iloc[:, 13].values

# Convert event dates to datetime objects (handling potential NaT from filtering)
Events = pd.to_datetime(Events.values.flatten(), format='%d-%m-%Y', errors='coerce')

# Preallocate array for weekday dummies (Monday to Thursday)
dow = np.full((len(DatesReturn), 4), np.nan)

# Create dummy variables for weekdays (Monday=0, Tuesday=1, ..., Thursday=3 in pandas weekday)
for d in range(4): # Loop for Monday (0) to Thursday (3)
    for i in range(len(DatesReturn)):
        dow[i, d] = 1 if DatesReturn[i].weekday() == d else 0

# Create events dummy variables
# Check if each date in DatesReturn is present in the filtered Events dates
Match = np.isin(DatesReturn, Events)

# Initialize event dummy matrix. Adding extra rows initially to handle shifting.
# Shape: (number of return dates + fD, fD)
EDummy = np.zeros((len(DatesReturn) + fD, fD))

# Create dummies for event days and subsequent days (up to fD)
for i in range(len(DatesReturn)):
    if Match[i]: # If the current date is an event date
        # Set dummies for the next fD days
        for fd_idx in range(fD):
            if (i + 1 + fd_idx) < (len(DatesReturn) + fD): # Ensure index is within bounds
                EDummy[i + 1 + fd_idx, fd_idx] = 1

# Remove the first fD rows as they cannot have event dummies referring to previous events
EDummy = EDummy[fD:, :]

# Create a dummy variable for tax days (Jan 1-5)
T = np.zeros(len(Ret))
dday = DatesReturn.dt.day
dmonth = DatesReturn.dt.month

# Identify dates that are in January and between the 1st and 5th
T[(dmonth == 1) & (dday >= 1) & (dday <= 5)] = 1

# --- Prepare Regression Data ---
# Create lagged returns matrix
# np.roll shifts the array. We roll by lag and set the first 'lag' elements to NaN.
RetLag = np.column_stack([np.roll(Ret, lag) for lag in range(1, maxlag + 1)])
RetLag[:maxlag, :] = np.nan # Set the initial lagged values to NaN


## 4. Estimate regression model

The regression model reads as:
$$
r_t = \delta_0
+ \sum_{i=1}^{3} \delta_{1,i} r_{t-i}
+ \sum_{i=1}^{4} \delta_{2,i} D_{i,t}
+ \delta_3 T_t
+ \sum_{i=1}^{3} \delta_{4,i} E_{i,t}
+ \varepsilon_t
$$
where:
- $r_t$ is the return of the test asset
- $r_{t-i}$ are lagged returns at time $t-i$
- $D_{i,t}$ is a dummy controlling for the day of the week, with $i=\{Monday, Tuesday, Wednesday, Thursday\}$
- $T_t$ is a dummy controlling for the first five days of the taxation year
- $E_{i,t}$ is the set of event dummies capturing the effect of the event and following days


Additionally, we compute HAC corrected standard errors, up to four lags.

In [13]:
# Stack lagged returns, weekday dummies, tax day dummy, and event dummies
X = np.column_stack([RetLag, dow, T, EDummy[:, :fD]])

# Remove rows with NaN values (due to lagged returns) from both X and the dependent variable (Ret)
# This aligns the data for regression
X = X[maxlag:]
Ret_trimmed = Ret[maxlag:]

# Adding a constant for the intercept in statsmodels OLS
X = sm.add_constant(X)

# --- Fit the Linear Regression Model ---
mdl = sm.OLS(Ret_trimmed, X).fit(
    cov_type='HAC',
    cov_kwds={'maxlags': 4}
)

# Extract coefficients and standard errors
coeff = np.round(mdl.params, 5)
se = mdl.bse  # Standard errors

# Calculate t-statistics (handle division by zero if se is 0)
Sig = np.divide(coeff, se, out=np.full_like(coeff, np.nan), where=se!=0)

# Combine coefficients and t-statistics for display
res = np.vstack([coeff, Sig])


## 5. Save data


In [12]:
# Names for lagged returns
rString = [f"R_t-{lag}" for lag in range(1, maxlag + 1)]
# Names for weekday dummies
varnames = ["Mon", "Tue", "Wed", "Thu", "TaxDays"]
# Names for event dummies
eString = [f"Event +{fd_idx + 1}" for fd_idx in range(fD)]
# All variable names, including the constant (added by statsmodels)
var = ["Constant"] + rString + varnames + eString

# Create a pandas DataFrame to display the regression results
tab = pd.DataFrame(res, columns=var, index=['Coefficient', 't-statistic'])

# --- Display and Save Results ---
print("Regression Analysis Results:")
print(tab)

# Write the results table to an Excel file based on the America flag
if America == 0:
    tab.to_excel('RegressionAnalysis_All.xlsx')
else:
    tab.to_excel('RegressionAnalysis_America.xlsx')


Regression Analysis Results:
             Constant      R_t-1     R_t-2     R_t-3      Mon       Tue  \
Coefficient  0.083900   0.174470 -0.019240  0.029650 -0.20031 -0.040430   
t-statistic  3.917134  11.837711 -1.241551  1.956743 -6.10583 -1.297666   

                  Wed      Thu  TaxDays  Event +1  Event +2  Event +3  
Coefficient  0.032420 -0.05500  0.25682 -0.684740  0.668510  0.008280  
t-statistic  1.054002 -1.83041  2.08305 -1.668165  2.282012  0.033866  
